# Homework: Introduction

## Q1. Running Elastic

What's the version.build_hash value?

`42f05b9372a9a4a470db3b52817899b99a76ee73`

In [6]:
!curl localhost:9200

{
  "name" : "cc176d2867a3",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "W81LUyvVRa-F-rWZ77l_jw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Which function do you use for adding your data to elastic?

`insert`

## Q3. Searching
What's the score for the top ranking result?
`84.05`

In [8]:
from elasticsearch import Elasticsearch
index_name = "course-questions"
es_client = Elasticsearch('http://localhost:9200') 

In [9]:
query = "How do I execute a command in a running docker container?"

In [10]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [11]:
max(elastic_search(query))

84.050095

## Q4. Filtering

Return 3 results. What's the 3rd question returned by the search engine?

`'How do I copy files from a different folder into docker container’s working directory?'`

In [12]:
def elastic_search_filtered(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [13]:
elastic_search_filtered(query)[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

What's the length of the resulting prompt? (use the len function)

962
`1462`
1962
2462


In [14]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"Q: {doc['question']}\A: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
search_results = elastic_search_filtered(query)
prompt = build_prompt(query, search_results)

In [17]:
len(prompt)

1462

## Q6. Tokens

Use the encode function. How many tokens does our prompt have?

122
222
322
422

In [2]:
%pip install tiktoken==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 998.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.2 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 19.5 MB/s eta 0:00:00m eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [45]:
! pip list

Package                  Version
------------------------ ----------
annotated-types          0.7.0
anyio                    4.4.0
archspec                 0.2.3
boltons                  23.0.0
Brotli                   1.0.9
certifi                  2024.7.4
cffi                     1.16.0
charset-normalizer       2.0.4
conda                    24.5.0
conda-libmamba-solver    24.1.0
conda-package-handling   2.2.0
conda_package_streaming  0.9.0
cryptography             42.0.5
distro                   1.9.0
filelock                 3.15.4
frozendict               2.4.2
fsspec                   2024.6.1
h11                      0.14.0
httpcore                 1.0.5
httpx                    0.27.0
idna                     3.7
Jinja2                   3.1.4
jsonpatch                1.33
jsonpointer              2.1
libmambapy               1.5.8
llvmlite                 0.43.0
MarkupSafe               2.1.5
menuinst                 2.0.2
more-itertools           10.3.0
mpmath               

In [18]:
import tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [22]:
len(encoding.encode(prompt))

322